In [ ]:
import json
from webui.utils import ObjectNamespace
from webui.chat import Character
import sounddevice as sd
from llama_cpp import Llama, LlamaGrammar

def chat(char,prompt="Continue the story without me"):
    full_response = ""
    for response in char.generate_text(prompt):
        full_response += response
    audio = char.text_to_speech(full_response)
    if audio: sd.play(*audio)            
    char.messages.append({"role": char.user, "content": prompt}) #add user prompt to history
    char.messages.append({
        "role": char.name,
        "content": full_response,
        "audio": audio
    })
    return full_response

def get_function(char, query, threshold=1.):
    results = char.ltm.get_query(query=query,include=["metadatas", "distances"],type="function",threshold=threshold)
    if len(results): # found function
        metadata = results[0]["metadata"]
        return metadata
    else: return None

def get_args(char, arguments, template, prompt, retries=3):
    response = char.LLM(f"""
              <|im_start|>system
              Complete these arguments ({arguments}) using this template ({template}) while following the user's request.<|im_end|>
              <|im_start|>user
              {prompt}<|im_end|>
              """,grammar=grammar,stop=["<|im_start|>","<|im_end|>"])
    try:
        args = json.loads(response["choices"][0]["text"])
    except Exception as e:
        print(e)
        args = get_args(char, arguments, template, prompt, retries-1) if retries>0 else None
    return args        

grammar = LlamaGrammar.from_file("./models/LLM/json.gbnf")

In [ ]:
"""
This is a demo to showcase function calling
"""
character = Character(
    character_file="./models/Characters/Amaryllis.json",
    model_file="./models/LLM/mistral-7b-openorca.Q4_K_M.gguf"
)
character.LLM = Llama(
    character.model_file,
    n_ctx=character.model_data["params"]["n_ctx"],
    n_gpu_layers=character.model_data["params"]["n_gpu_layers"],
    verbose=True
)
character.loaded=True
character

In [ ]:
"""
We first define the list of function definitions and put them to a vector database.
The description should be potential user prompts that can trigger the function.

arguments: list of argument names that the function uses
**kwargs: dictionary of [type: data type, description: description of argument, required: whether it is required]
"""
functions = [
    ObjectNamespace(
        description = "Can you please play a song for me?|I would love to hear some music.|Can you choose a song for me?Play a song, please.|Let’s enjoy some music together",
        function = "play_song",
        arguments = ["name","search"],
        name = {"type": "str", "description": "name of the song", "required": True},
        search = {"type": "boolean", "description": "search for the song if it doesn't exist"},
    ),
]

for data in functions:
    character.ltm.add_function(**data)

In [ ]:
prompt = "Can you play the song despacito for me?"
metadata = get_function(character,prompt,threshold=1.)
print(f"{metadata=}")
args = get_args(character, metadata['arguments'], metadata['template'], prompt)
args

In [ ]:
def play_song(name,search): # dummy function
    print(f"playing {name}. search song on youtube if it doesn't exist: {search}")
eval(metadata["function"])(**args) # evaluate the song